In [1]:
import json
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd
from selenium.webdriver.common.by import By

In [2]:
# wait
def wait(driver, selector_name):
    while (True):
        try:
            myElem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, selector_name)))
            if myElem is not None:
                break
        except TimeoutException:
            break

In [3]:
# scroll
def scroll(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
my_url = "https://soundcloud.com/discover"

In [ ]:
driver = webdriver.Chrome()
driver.get(my_url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
client_id = soup.find("div", {"id": "g_id_intermediate_iframe"}).get("data-src")
client_id = ((client_id.split('?')[1]).split('&')[0]).split('=')[1]

In [ ]:
client_id

In [ ]:
set_link = []

In [ ]:
driver = webdriver.Chrome()
driver.get(my_url)
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
link = soup.find_all("a", {
    "class": "sc-link-secondary sc-link-light playableTile__usernameHeading audibleTile__usernameHeading sc-truncate sc-text-h4"})

for item in link:
    user_url = "https://soundcloud.com" + item.get("href")
    if user_url not in set_link:
        set_link.append(user_url)

for user_url in set_link:
    if len(set_link) > 25:
        break;
    driver.get(user_url)
    wait(driver, 'profileHeaderInfo__userName')
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    user_name = soup.find("h2", {"class": "profileHeaderInfo__userName"})

    # Get followers
    follower_url = user_url + "/following"
    driver.get(follower_url)

    #Wait
    scroll(driver)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    followers = soup.find_all("a", {"class": "userBadgeListItem__heading"})
    for follower in followers:
        if follower not in set_link:
            set_link.append("https://soundcloud.com" + follower.get("href"))

In [ ]:
len(set_link)

In [ ]:
def get_all_user_data(set_link):
    #Get user id in SoundCloud/Discover by get meta tag in HEAD html
    set_data_user = []
    for link_user in set_link:
        driver = webdriver.Chrome()
        driver.get(link_user)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        #Get user id & user name & user url
        id_user = soup.find("meta", property="twitter:app:url:googleplay")
        set_data_user.append(id_user["content"].strip("soundcloud://users:"))

    return set_data_user

In [ ]:
set_user = get_all_user_data(set_link)

In [ ]:
len(set_user)

In [ ]:
#============ using API to get USER.csv =================
#Get set_user API
def get_user_by_API(client_id):
    set_data_user = []
    # Select first column of the dataframe as a series
    for item in set_user:
        one_data_user = []
        user_id = item
        api_url = f'https://api-v2.soundcloud.com/users/{user_id}?client_id={client_id}'
        # get request
        r = requests.get(api_url)
        # check whether it get the url
        # or if it got request, does it have a string "incomplete_results":true in the response it recieved?
        while r.status_code != requests.codes.ok or '"incomplete_results":true' in r:
            time.sleep(1)
            r = requests.get(api_url)
        #JSON processing
        json_str = json.loads(r.text)
        one_data_user.append(str(json_str['id']))
        one_data_user.append(str(json_str['username']))
        one_data_user.append(str(json_str['permalink_url']))
        set_data_user.append(one_data_user)

    return set_data_user

In [ ]:
get_user_by_API(client_id)

In [ ]:
def getTracksAPI(set_user, client_id, limit=100):
    set_tracks = set()
    temp1 = 'https://api-v2.soundcloud.com/users/'
    temp2 = f'/tracks?client_id={client_id}&limit={limit}'

    for item in set_user:
        string_tracks = ''
        api = temp1 + item + temp2
        while True:
            r = requests.get(api)
            if r.status_code != 200:
                time.sleep(5.0)
            else:
                break
        for temp in r.json()['collection']:
            string_tracks = string_tracks + str(temp['id']) + ','
        tup = (item, string_tracks)
        set_tracks.add(tup)
    return set_tracks

In [ ]:
getTracksAPI(set_user, client_id)

In [11]:
def createNoneObj(n):
    return ['None' for i in range(n)]

In [16]:
def getPlaylistsAPI(set_user, client_id, limit=100):
    playlists = []
    
    for user_id in set_user:
        
        url = f'https://api-v2.soundcloud.com/users/{user_id}/playlists?client_id={client_id}&limit={limit}'
        r = requests.get(url)

        while r.status_code != requests.codes.ok or '"incomplete_results":true' in r:
            time.sleep(1)
            print('wait')
            r = requests.get(url)

        y = json.loads(r.text)
        if len(y['collection']) == 0:
            playlists.append([user_id]+createNoneObj(24))
        else:
            for playlist in y['collection']:
                trackIDs = []
                for track in playlist['tracks']:
                    trackIDs.append(str(track['id']))
                allTrackIDs = ','.join(trackIDs)
                playlists.append([playlist['user_id'],
                                 playlist['artwork_url'],
                                 playlist['created_at'],
                                 playlist['duration'],
                                 playlist['embeddable_by'],
                                 playlist['genre'],
                                 playlist['id'],
                                 playlist['kind'],
                                 playlist['label_name'],
                                 playlist['last_modified'],
                                 playlist['license'],
                                 playlist['likes_count'],
                                 playlist['managed_by_feeds'],
                                 playlist['permalink'],
                                 playlist['permalink_url'],
                                 playlist['public'],
                                 playlist['reposts_count'],
                                 playlist['sharing'],
                                 playlist['tag_list'],
                                 playlist['title'],
                                 playlist['uri'],
                                 playlist['is_album'],
                                 playlist['published_at'],
                                 playlist['display_date'],
                                 allTrackIDs])

    return playlists

In [19]:
temp = getPlaylistsAPI(['293713798'], 'FjnXkiGFvyaVIYtXadMm9pqIDawoxzUW')

wait


In [20]:
f_out = pd.DataFrame(temp, columns=['user_id',
                                 'artwork_url',
                                 'created_at',
                                 'duration',
                                 'embeddable_by',
                                 'genre',
                                 'id',
                                 'kind',
                                 'label_name',
                                 'last_modified',
                                 'license',
                                 'likes_count',
                                 'managed_by_feeds',
                                 'permalink',
                                 'permalink_url',
                                 'public',
                                 'reposts_count',
                                 'sharing',
                                 'tag_list',
                                 'title',
                                 'uri',
                                 'is_album',
                                 'published_at',
                                 'display_date',
                                 'allTrackIDs'])
f_out.to_csv('playlistApi.csv', index=False, sep='\t')
